Install and load packages

In [ ]:
install.packages('readxl')
install.packages('dplyr')
install.packages('ggplot2')
install.packages('stargazer')
install.packages('purrr')
install.packages('stringr')
install.packages('lessR')
install.packages('fixest')
install.packages('ordinal')
install.packages('effsize')


In [ ]:
library(readxl)
library(dplyr)
library(ggplot2)
library(stats)
library(stargazer)
library(purrr)
library(stringr)
library(tidyverse)
library(lessR)
library(fixest)
library(sandwich)
library(lmtest)
library(car)
library(effsize)


Load df

In [ ]:
# create a list of file names and corresponding is_editorial values
file_names <- c("df_rap_final.xlsx", "df_pop_final.xlsx", "df_variete_final.xlsx", "df_ete_final.xlsx",
                "df_latino_final.xlsx", "df_ambiance_final.xlsx", "df_electro_final.xlsx", "df_rock_final.xlsx",
                "df_indie_final.xlsx", "df_sport_final.xlsx", "df_rnb_final.xlsx", "df_afro_final.xlsx",
                "df_kpop_final.xlsx", "df_detente_final.xlsx", "df_dormir_final.xlsx", "df_soiree_final.xlsx",
                "df_zouk_final.xlsx", "df_reggae_final.xlsx", "df_maison_final.xlsx", "df_decennies_final.xlsx",
                "df_alternatif_final.xlsx", "df_amour_final.xlsx", "df_metal_final.xlsx", "df_jazz_final.xlsx",
                "df_tendances_final.xlsx", "df_equal_final.xlsx", "df_classique_final.xlsx", "df_folk_final.xlsx",
                "df_focus_final.xlsx", "df_soul_final.xlsx", "df_netflix_final.xlsx", "df_enfants_final.xlsx",
                "df_etudiants_final.xlsx", "df_bienetre_final.xlsx", "df_punk_final.xlsx", "df_ambient_final.xlsx",
                "df_country_final.xlsx", "df_blues_final.xlsx", "df_arabe_final.xlsx", "df_voyage_final.xlsx",
                "df_frequency_final.xlsx"
                )


type_values <- c('genre', 'genre', 'genre', 'mood', 'genre', 'mood', 'genre', 'genre', 'genre', 'mood', 'genre',
                 'genre', 'genre', 'mood', 'mood', 'mood', 'genre', 'genre', 'mood', 'trend', 'genre', 'mood',
                 'genre', 'genre', 'trend', 'trend', 'genre', 'genre', 'mood', 'genre', 'trend', 'mood', 'mood',
                 'mood', 'genre', 'genre', 'genre', 'genre', 'genre', 'mood', 'trend'
                 )

df_list <- list()
for (i in seq_along(file_names)) {
  file_name <- file_names[i]
  type <- type_values[i]
  df <- read_xlsx(file_name)
  df$type <- type
  df_list[[i]] <- df
}

df <- Reduce(function(x, y) merge(x, y, all = TRUE), df_list)


Data management

Rao-Stirling

In [ ]:
# Extract numeric values
df$diversity_clean <- as.numeric(gsub("[^[:digit:].-]", "", df$stirling))
df$diversity_clean_2 <- as.numeric(gsub("[^[:digit:].-]", "", df$stirling_2))

# Print numeric values
df$diversity_clean
df$diversity_clean_2

# Scale values from 0 to 1
df$diversity_norm <- rescale(diversity_clean, df, kind='z')
df$diversity_norm_2 <- rescale(diversity_clean_2, df, kind='z')

# Print scaled values
df$diversity_norm
df$diversity_norm_2


HHI

In [ ]:
# Scale values from 0 to 1
df$hhi_norm <- rescale(hhi, df, kind='z')
df$hhi_norm_2 <- rescale(hhi_2, df, kind='z')

# Print scaled values
df$hhi_norm
df$hhi_norm_2


Dist

In [ ]:
# Scale values from 0 to 1
df$dist <- rescale(mean_distance, df, kind='z')
df$dist_2 <- rescale(mean_distance_2, df, kind='z')

# Print scaled values
df$dist
df$dist_2


Covariates

In [ ]:
# Scale covariates and log followers
df$track_pop_norm <- rescale(as.numeric(track_pop), df, kind='0to1')
df$track_pop_norm

df$artist_pop_norm <- rescale(as.numeric(artist_pop), df, kind='0to1')
df$artist_pop_norm

df$nb_tracks_res <- rescale(as.numeric(nb_tracks), df, kind='z')
df$nb_tracks_norm <- rescale(nb_tracks_res, df, kind='0to1')
df$nb_tracks_norm

df$nb_artists_norm <- rescale(as.numeric(nb_artists), df, kind='0to1')
df$nb_artists_norm

df$nb_clusters_norm <- rescale(as.numeric(nb_clusters), df, kind='0to1')
df$nb_clusters_norm

df$nb_clusters_2_norm <- rescale(as.numeric(nb_clusters_2), df, kind='0to1')
df$nb_clusters_2_norm


In [ ]:
summary(df$playlist_followers)
df$followers_class <- case_when(df$playlist_followers < 27701 ~ "low",
                                df$playlist_followers >= 27701 & df$playlist_followers < 145454 ~ "mid_low",
                                df$playlist_followers >= 145454 & df$playlist_followers < 492824 ~ "mid_high",
                                df$playlist_followers >= 492824 ~ "high")
table(df$followers_class)


Descriptive stats

Summary stats

In [ ]:
nrow(df)
sum(df$nb_tracks)


In [ ]:
summary(df$diversity_clean)
summary(df$diversity_clean_2)


In [ ]:
summary(df$nb_clusters)
summary(df$nb_clusters_2)
summary(df$hhi)
summary(df$hhi_2)
summary(df$mean_distance)
summary(df$mean_distance_2)


In [ ]:
table(df$is_editorial)


In [ ]:
tapply(df$nb_clusters, df$is_editorial, mean)
tapply(df$nb_clusters_2, df$is_editorial, mean)
tapply(df$hhi, df$is_editorial, mean)
tapply(df$hhi_2, df$is_editorial, mean)
tapply(df$mean_distance, df$is_editorial, mean)
tapply(df$mean_distance_2, df$is_editorial, mean)
tapply(df$diversity_clean, df$is_editorial, mean)
tapply(df$diversity_clean_2, df$is_editorial, mean)


In [ ]:
table(df$type)


In [ ]:
tapply(df$nb_clusters, df$type, mean)
tapply(df$nb_clusters_2, df$type, mean)
tapply(df$hhi, df$type, mean)
tapply(df$hhi_2, df$type, mean)
tapply(df$mean_distance, df$type, mean)
tapply(df$mean_distance_2, df$type, mean)
tapply(df$diversity_clean, df$type, mean)
tapply(df$diversity_clean_2, df$type, mean)


In [ ]:
sapply(df[, c('nb_tracks', 'playlist_followers', 'track_pop', 'artist_pop')], function(x) c(summary(x), sd(x)))


Barplots

In [ ]:
ggplot(df, aes(x = diversity_clean)) + 
  geom_histogram(binwidth = 0.025, color = "black", fill = "#a7a7f9") +
  xlim(0, 0.5) +
  theme_bw() +
  theme(panel.grid.major.y = element_line(linetype = "dotted"),
        panel.grid.minor.y = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        aspect.ratio = 0.8) +
  labs(x = 'k\U03b1 Rao-Stirling', y = 'Frequency')


In [ ]:
ggplot(df, aes(x = diversity_clean_2)) + 
  geom_histogram(binwidth = 0.025, color = "black", fill = "#a7a7f9") +
  xlim(0, 0.7) +
  theme_bw() +
  theme(panel.grid.major.y = element_line(linetype = "dotted"),
        panel.grid.minor.y = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        aspect.ratio = 0.8) +
  labs(x = 'k\U03b2  Rao-Stirling', y = 'Frequency')


Radar charts

In [ ]:
# Adjust options to display all columns of the data frame
options(width = 10000)

# Convert columns 15 and 16 to numeric
df_3[, 15:16] <- lapply(df_3[, 15:16], as.numeric)

# Create a new data frame to store scaled values
scaled_df <- data.frame(df_3)

# Rescale loudness, tempo, and duration to the range 0-1
scaled_df$loudness <- scale(scaled_df$loudness, center = min(scaled_df$loudness), scale = max(scaled_df$loudness) - min(scaled_df$loudness))
scaled_df$tempo <- scale(scaled_df$tempo, center = min(scaled_df$tempo), scale = max(scaled_df$tempo) - min(scaled_df$tempo))
scaled_df$duration_ms <- scale(scaled_df$duration_ms, center = min(scaled_df$duration_ms), scale = max(scaled_df$duration_ms) - min(scaled_df$duration_ms))

# Subset your scaled dataframe to select the tracks you want to compare
selected_tracks <- scaled_df[scaled_df$track_name %in% c("I Feel It Coming", "November Rain"), ]

# Print the values for the selected tracks
for (track_name in unique(selected_tracks$track_name)) {
  track_data <- selected_tracks[selected_tracks$track_name == track_name, 7:16]
  cat("Track:", track_name, "\n")
  print(track_data)
  cat("\n")
}


OLS models

Robusts OLS

In [ ]:
# Fit your robust linear regression 
lm_1 <- lm(diversity_norm ~ followers_class + nb_tracks + track_pop + type, data = df)
lm_1_1 <- lm(diversity_norm ~ type, data = df)

lm1_robust_clustered <- coeftest(lm_1, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
lm1_1_robust_clustered <- coeftest(lm_1_1, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
print(lm1_robust_clustered)

lm_2 <- lm(diversity_norm_2 ~ followers_class + nb_tracks + track_pop + type, data = df)
lm_2_1 <- lm(diversity_norm_2 ~ type, data = df)

lm2_robust_clustered <- coeftest(lm_2, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
lm2_1_robust_clustered <- coeftest(lm_2_1, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
print(lm2_robust_clustered)

stargazer(lm1_1_robust_clustered, lm1_robust_clustered, lm2_1_robust_clustered, lm2_robust_clustered)


Robust OLS Comparison 

In [ ]:
# Perform OLS HHI*
lm_3 <- lm(hhi_norm ~ followers_class + nb_tracks + track_pop + type, data = df)
lm_3_1 <- lm(hhi_norm ~ type, data = df)

lm3_robust_clustered <- coeftest(lm_3, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
lm3_1_robust_clustered <- coeftest(lm_3_1, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
print(lm3_robust_clustered)

# Perform OLS HHI**
lm_4 <- lm(hhi_norm_2 ~ followers_class + nb_tracks + track_pop + type, data = df)
lm_4_1 <- lm(hhi_norm_2 ~ type, data = df)

lm4_robust_clustered <- coeftest(lm_4, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
lm4_1_robust_clustered <- coeftest(lm_4_1, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
print(lm4_robust_clustered)

stargazer(lm3_1_robust_clustered, lm3_robust_clustered, lm4_1_robust_clustered, lm4_robust_clustered)


In [ ]:
# Perform OLS Dist*
lm_5 <- lm(dist ~ followers_class + nb_tracks + track_pop + type, data = df)
lm_5_1 <- lm(dist ~ type, data = df)

lm5_robust_clustered <- coeftest(lm_5, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
lm5_1_robust_clustered <- coeftest(lm_5_1, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
print(lm5_robust_clustered)

# Perform OLS Dist**
lm_6 <- lm(dist_2 ~ followers_class + nb_tracks + track_pop + type, data = df)
lm_6_1 <- lm(dist_2 ~ type, data = df)

lm6_robust_clustered <- coeftest(lm_6, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
lm6_1_robust_clustered <- coeftest(lm_6_1, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
print(lm6_robust_clustered)

stargazer(lm5_1_robust_clustered, lm5_robust_clustered, lm6_1_robust_clustered, lm6_robust_clustered)


In [ ]:
stargazer(lm1_1_robust_clustered, lm1_robust_clustered, lm3_1_robust_clustered, lm3_robust_clustered, lm5_1_robust_clustered, lm5_robust_clustered)


In [ ]:
stargazer(lm_2_1, lm_2, lm_4_1, lm_4, lm_6_1, lm_6)

In [ ]:
stargazer(lm2_1_robust_clustered, lm2_robust_clustered, lm4_1_robust_clustered, lm4_robust_clustered, lm6_1_robust_clustered, lm6_robust_clustered)

Robustness checks

Control for clusters

In [ ]:
# Perform OLS
lm_1_2 <- lm(diversity_norm ~ followers_class + nb_tracks + track_pop + type + as.numeric(nb_clusters), data = df)
lm1_2_robust_clustered <- coeftest(lm_3, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
summary(lm_1_2)

# Perform OLS
lm_2_2 <- lm(diversity_norm_2 ~ followers_class + nb_tracks + track_pop + type + as.numeric(nb_clusters_2), data = df)
lm2_2_robust_clustered <- coeftest(lm_4, vcov = vcovCL, type = "HC1", df = 2, cluster = ~type)
summary(lm_2_2)

stargazer(lm1_robust_clustered, lm1_2_robust_clustered, lm2_robust_clustered, lm2_2_robust_clustered)


Multicolinearity

In [ ]:
vif(lm_1)
vif(lm_2)


Test on silhouette scores

In [ ]:
# create a list of file names and corresponding is_editorial values
file_names <- c("robust_afro.xlsx", "robust_alternatif.xlsx", "robust_ambiance.xlsx", "robust_ambient.xlsx",
                "robust_amour.xlsx", "robust_arabe.xlsx", "robust_bienetre.xlsx", "robust_blues.xlsx",
                "robust_classique.xlsx", "robust_country.xlsx", "robust_decennies.xlsx", "robust_detente.xlsx",
                "robust_dormir.xlsx", "robust_electro.xlsx", "robust_enfants.xlsx", "robust_equal.xlsx",
                "robust_ete.xlsx", "robust_etudiants.xlsx"
                )

df_list <- list()
for (i in seq_along(file_names)) {
  file_name <- file_names[i]
  df <- read_xlsx(file_name)
  df_list[[i]] <- df
}

df_robust <- Reduce(function(x, y) merge(x, y, all = TRUE), df_list)


In [ ]:
# Perform paired t-tests
t_test_5_6 <- t.test(df_robust$silhouette_score_5, df_robust$silhouette_score_6, paired = TRUE)
cohen_d_5_6 <- cohen.d(df_robust$silhouette_score_5, df_robust$silhouette_score_6, hedges.correction = TRUE)
t_test_5_6
cohen_d_5_6

t_test_5_4 <- t.test(df_robust$silhouette_score_5, df_robust$silhouette_score_4, paired = TRUE)
cohen_d_5_4 <- cohen.d(df_robust$silhouette_score_5, df_robust$silhouette_score_4, hedges.correction = TRUE)
t_test_5_4
cohen_d_5_4

t_test_5_6_2 <- t.test(df_robust$silhouette_score_5_2, df_robust$silhouette_score_6_2, paired = TRUE)
cohen_d_5_6_2 <- cohen.d(df_robust$silhouette_score_5_2, df_robust$silhouette_score_6_2, hedges.correction = TRUE)
t_test_5_6_2
cohen_d_5_6_2

t_test_5_4_2 <- t.test(df_robust$silhouette_score_5_2, df_robust$silhouette_score_4_2, paired = TRUE)
cohen_d_5_4_2 <- cohen.d(df_robust$silhouette_score_5_2, df_robust$silhouette_score_4_2, hedges.correction = TRUE)
t_test_5_4_2
cohen_d_5_4_2
